In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from numpy.random import randn
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

pd.set_option('display.max_columns', None)

df = pd.read_csv('dataset/diabetes_binary_health_indicators_BRFSS2021.csv')
# Assuming df is your DataFrame and "Diabetes_binary" is your target column
class_0 = df[df.Diabetes_binary == 0]
class_1 = df[df.Diabetes_binary == 1]

# Downsample class 0 to match the number of instances in class 1
class_0_downsampled = class_0.sample(len(class_1))

# Concatenate the downsampled class 0 with class 1
data = pd.concat([class_0_downsampled, class_1], axis=0)
print (data.shape)
print (data.tail())
print (data.columns)

(67136, 22)
        Diabetes_binary  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
236343              1.0       1       0.0          1  27.0     0.0     0.0   
236355              1.0       1       1.0          1  26.0     0.0     0.0   
236362              1.0       1       1.0          1  32.0     0.0     0.0   
236369              1.0       1       1.0          1  33.0     0.0     0.0   
236373              1.0       1       1.0          1  21.0     0.0     0.0   

        HeartDiseaseorAttack  PhysActivity  Fruits  Veggies  \
236343                   0.0             1       1        1   
236355                   0.0             0       1        1   
236362                   1.0             1       0        0   
236369                   0.0             0       0        1   
236373                   0.0             1       1        1   

        HvyAlcoholConsump  AnyHealthcare  NoDocbcCost  GenHlth  MentHlth  \
236343                  0              1          0.0      3.0  

In [2]:
features = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 
                 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 
                 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 
                 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']
label = ['Diabetes_binary']
X = data[features]
y = data[label] 

In [3]:
X_true_train, X_true_test, y_true_train, y_true_test = train_test_split(X, y, test_size=0.30, random_state=42)
clf_true = RandomForestClassifier(n_estimators=100)
clf_true.fit(X_true_train,y_true_train)
y_true_pred=clf_true.predict(X_true_test)
print("Base Accuracy:",metrics.accuracy_score(y_true_test, y_true_pred))
print("Base classification report:",metrics.classification_report(y_true_test, y_true_pred))

c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Base Accuracy: 0.732485973884117
Base classification report:               precision    recall  f1-score   support

         0.0       0.75      0.70      0.72     10042
         1.0       0.72      0.77      0.74     10099

    accuracy                           0.73     20141
   macro avg       0.73      0.73      0.73     20141
weighted avg       0.73      0.73      0.73     20141



In [4]:
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

In [5]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = generator.predict(x_input)
    y = np.zeros((n_samples, 1))
    
    return X, y

In [6]:
# generate n real samples with class labels; We randomly select n samples from the real data
def generate_real_samples(n):
    X = data.sample(n)
    y = np.ones((n, 1))
    return X, y

In [7]:
def define_generator(latent_dim, n_outputs=22):
    model = Sequential()
    model.add(Dense(15, activation='relu',  kernel_initializer='he_uniform', input_dim=latent_dim))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(n_outputs, activation='linear'))
    return model

In [8]:
generator1 = define_generator(10, 22)
generator1.summary()

c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 15)             │           165 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 30)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 22)             │           682 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,327 (5.18 KB)

 Trainable params: 1,327 (5.18 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
def define_discriminator(n_inputs=22):
    model = Sequential()
    model.add(Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=n_inputs))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [10]:
discriminator1 = define_discriminator(22)
discriminator1.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 25)             │           575 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 50)             │         1,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,926 (7.52 KB)

 Trainable params: 1,926 (7.52 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # compile model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [12]:
# create a line plot of loss for the gan and save to file
def plot_history(d_hist, g_hist):
    # plot loss
    plt.subplot(1, 1, 1)
    plt.plot(d_hist, label='d')
    plt.plot(g_hist, label='gen')
    plt.show()
    plt.close()

In [13]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=5000, n_batch=128, n_eval=200):
    # determine half the size of one batch, for updating the  discriminator
    half_batch = int(n_batch / 2)
    d_history = []
    g_history = []
    # manually enumerate epochs
    for epoch in range(n_epochs):
    
      # prepare real samples
      x_real, y_real = generate_real_samples(half_batch)
      # prepare fake examples
      x_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
      # update discriminator
      d_loss_real, d_real_acc = d_model.train_on_batch(x_real, y_real)
      d_loss_fake, d_fake_acc = d_model.train_on_batch(x_fake, y_fake)
      d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
      # prepare points in latent space as input for the generator
      x_gan = generate_latent_points(latent_dim, n_batch)
      # create inverted labels for the fake samples
      y_gan = np.ones((n_batch, 1))
      # update the generator via the discriminator's error
      g_loss_fake = gan_model.train_on_batch(x_gan, y_gan)
      print('EPOCH: ' + str(epoch+1))
      d_history.append(d_loss)
      g_history.append(g_loss_fake)
    plot_history(d_history, g_history)
    g_model.save('trained_generated_model3.h5')

In [14]:
# size of the latent space
latent_dim = 10
# create the discriminator
discriminator = define_discriminator()
# create the generator
generator = define_generator(latent_dim)
# create the gan
gan_model = define_gan(generator, discriminator)
# train model
train(generator, discriminator, gan_model, latent_dim)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  


c:\Users\ensar\Desktop\Diabetes-Prediction\venv\lib\site-packages\keras\src\backend\tensorflow\trainer.py:71: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


EPOCH: 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
EPOCH: 2
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 3
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 4
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 6
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
EPOCH: 7
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 8
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 9
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
EPOCH: 10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 11
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 12
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 13
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 14
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 15
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 16
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
EPOCH: 17
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
EPOCH: 18
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 19
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
EPOCH: 20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
EPOCH: 21
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
E

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
model =load_model('trained_generated_model2.h5')

latent_points = generate_latent_points(10, 750)
X = model.predict(latent_points)
data_fake = pd.DataFrame(data=X,  columns=['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 
                 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 
                 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 
                 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income'])
data_fake.head()

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,-9.500153,-1.798378,-9.403749,-3.708908,7.053625,-4.982277,-1.375199,2.169758,1.104635,-11.200621,...,-4.973795,-4.811009,11.917920,-3.505525,10.897144,13.403828,-1.722203,-6.586629,-8.181518,3.501793
1,-4.494378,-1.000350,-4.275302,-1.664019,2.595856,-1.846845,-0.616683,-0.049434,-0.200754,-5.813783,...,-2.840287,-2.183266,4.630520,-2.223422,4.981379,5.175343,-0.701862,-2.831196,-3.480281,1.764666
2,-4.730064,-1.935831,-6.137761,-2.750699,3.338793,-3.474374,-0.228971,0.635288,0.375237,-6.659486,...,-3.132330,-1.917413,6.018384,-1.501016,5.834258,7.066011,-1.939550,-3.383054,-3.990360,1.523848
3,-4.778231,-1.203619,-4.720156,-1.801063,3.765876,-3.061099,-1.634511,0.740718,-0.682719,-5.555139,...,-3.063214,-1.366896,4.979402,-1.495027,5.255156,6.017437,-1.438279,-2.766773,-3.295077,2.050755
4,-7.455038,-1.356716,-7.381846,-1.970566,5.907843,-4.007869,-1.555057,1.667246,0.401965,-8.696170,...,-4.025335,-4.138158,7.669865,-2.944604,8.620721,9.782400,-1.401259,-5.144302,-6.357512,3.058285


In [ ]:
binary_columns = ['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 
                 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 
                 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk', 'Sex']

for col in binary_columns:
    mean = data_fake[col].mean()
    data_fake[col] = data_fake[col] > mean
    data_fake[col] = data_fake[col].astype(int)

In [ ]:
features = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 
                 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies', 
                 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth', 
                 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income']
label = ['Diabetes_binary']
X_fake_created = data_fake[features]
y_fake_created = data_fake[label]

In [ ]:
X_fake_created.head(100)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,0,0,7.053625,0,0,1,1,0,1,1,0,0,11.917920,-3.505525,10.897144,1,1,-6.586629,-8.181518,3.501793
1,1,1,1,2.595856,1,1,0,0,1,1,0,1,1,4.630520,-2.223422,4.981379,0,1,-2.831196,-3.480281,1.764666
2,0,1,1,3.338793,1,1,0,1,1,0,0,1,1,6.018384,-1.501016,5.834258,0,0,-3.383054,-3.990360,1.523848
3,1,1,1,3.765876,1,0,1,0,1,1,0,1,1,4.979402,-1.495027,5.255156,0,1,-2.766773,-3.295077,2.050755
4,1,1,1,5.907843,1,0,1,1,1,1,1,1,0,7.669865,-2.944604,8.620721,0,1,-5.144302,-6.357512,3.058285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0,8.092458,0,0,0,0,0,0,1,0,0,13.647458,-5.088439,13.628555,1,0,-7.900844,-10.201932,5.050296
96,0,0,0,6.564524,0,0,0,0,0,0,1,0,0,12.446147,-4.042534,11.266113,1,0,-7.129158,-8.712857,3.539884
97,1,0,0,7.657236,0,0,1,1,0,1,1,0,0,9.822579,-3.299392,11.270383,1,1,-7.036613,-7.707760,3.406665
98,0,0,0,4.754378,1,1,0,0,0,0,0,0,0,8.473081,-2.556750,8.403027,0,1,-5.122649,-6.620308,2.757801


In [ ]:
y_fake_created.head()

,Diabetes_binary
0,1
1,1
2,1
3,1
4,1
...,...
95,1
96,1
97,1
98,1
